In [1]:
from torch_geometric.data import Data
from torch_geometric.nn import GATConv
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.loader import DataLoader
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool, global_add_pool, global_max_pool
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import pandas as pd
import os
import scipy.sparse as sp
import random

In [2]:
root = 'data_used'
label_list = os.listdir(root)
label_list

['ASD', 'HC']

In [13]:
from torch_geometric.utils import dense_to_sparse
class_dict = {
            "HC": 0,
            "ASD": 1,
            }
threshold = 0.2
data_processed = []
for label_files in label_list:
    label = torch.LongTensor([class_dict[label_files]])
    list = os.listdir(os.path.join(root, label_files))
    for files in list:
        subj_fc_dir = os.path.join(root, label_files, files)
        subj_mat_fc=np.loadtxt(subj_fc_dir)[:176,:90]
        print("reading data " + subj_fc_dir)
        subj_mat_fc_adj = np.corrcoef(np.transpose(subj_mat_fc))
        subj_mat_fc_adj = subj_mat_fc_adj - np.diag(np.diag(subj_mat_fc_adj))
        #take the upper triangle and compute the threshold
        subj_fc_adj_up=subj_mat_fc_adj[np.triu_indices(90,k=1)]
        subj_fc_adj_list = subj_fc_adj_up.reshape((-1))
        thindex = int(threshold * subj_fc_adj_list.shape[0])
        thremax = subj_fc_adj_list[subj_fc_adj_list.argsort()[-1 * thindex-1]]#
        #avoiding Nan
        subj_fc_adj_t = np.zeros((90, 90))
        subj_fc_adj_t[subj_mat_fc_adj > thremax] = 1
        subj_mat_fc_adj=subj_fc_adj_t
        fcedge_index, _ = dense_to_sparse(torch.from_numpy(subj_mat_fc_adj.astype(np.int16)))

        subj_mat_fc_list = subj_mat_fc.reshape((-1))
        subj_mat_fc_new = (subj_mat_fc - min(subj_mat_fc_list)) / (
                max(subj_mat_fc_list) - min(subj_mat_fc_list))
        # subj_mat_fc_new = (subj_mat_fc - np.mean(subj_mat_fc, axis=0, keepdims=True)) / np.std(subj_mat_fc, axis=0, keepdims=True)

        subj_mat_fc_new = np.transpose(subj_mat_fc_new)

        rowsum = np.array(subj_mat_fc_adj.sum(1))
        N = np.diag(rowsum)
        degree_C_BOLD=np.concatenate((N,subj_mat_fc_new),1)
        # BOLD_C_degree=np.concatenate((subj_mat_fc_new,N),1)
        ###one-hot###
        # subj_mat_fc_new=np.eye(args.num_nodes)
        data_processed.append(Data(x=torch.from_numpy(degree_C_BOLD).float(), edge_index=fcedge_index, y=torch.tensor(label)))

reading data data_used\ASD\CMU_a_0050649_rois_aal.1D
reading data data_used\ASD\Leuven_1_0050686_rois_aal.1D
reading data data_used\ASD\Leuven_1_0050689_rois_aal.1D
reading data data_used\ASD\Leuven_1_0050690_rois_aal.1D
reading data data_used\ASD\Leuven_1_0050693_rois_aal.1D
reading data data_used\ASD\Leuven_1_0050694_rois_aal.1D
reading data data_used\ASD\Leuven_1_0050695_rois_aal.1D
reading data data_used\ASD\Leuven_1_0050696_rois_aal.1D
reading data data_used\ASD\Leuven_1_0050697_rois_aal.1D
reading data data_used\ASD\Leuven_1_0050700_rois_aal.1D
reading data data_used\ASD\Leuven_1_0050702_rois_aal.1D


C:\Users\86189\AppData\Local\Temp\ipykernel_11620\1334165750.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data_processed.append(Data(x=torch.from_numpy(degree_C_BOLD).float(), edge_index=fcedge_index, y=torch.tensor(label)))


reading data data_used\ASD\Leuven_1_0050704_rois_aal.1D
reading data data_used\ASD\Leuven_1_0050705_rois_aal.1D
reading data data_used\ASD\Leuven_1_0050708_rois_aal.1D
reading data data_used\ASD\Leuven_1_0050711_rois_aal.1D
reading data data_used\ASD\Leuven_2_0050743_rois_aal.1D
reading data data_used\ASD\Leuven_2_0050745_rois_aal.1D
reading data data_used\ASD\Leuven_2_0050746_rois_aal.1D
reading data data_used\ASD\Leuven_2_0050747_rois_aal.1D
reading data data_used\ASD\Leuven_2_0050748_rois_aal.1D
reading data data_used\ASD\Leuven_2_0050749_rois_aal.1D
reading data data_used\ASD\Leuven_2_0050751_rois_aal.1D
reading data data_used\ASD\Leuven_2_0050752_rois_aal.1D
reading data data_used\ASD\Leuven_2_0050753_rois_aal.1D
reading data data_used\ASD\Leuven_2_0050754_rois_aal.1D
reading data data_used\ASD\Leuven_2_0050755_rois_aal.1D
reading data data_used\ASD\Leuven_2_0050756_rois_aal.1D
reading data data_used\ASD\Leuven_2_0050757_rois_aal.1D
reading data data_used\ASD\MaxMun_d_0051329_rois

In [14]:
import random
random.seed(1234)
random.shuffle(data_processed)
data_processed

[Data(x=[90, 266], edge_index=[2, 1602], y=[1]),
 Data(x=[90, 266], edge_index=[2, 1603], y=[1]),
 Data(x=[90, 266], edge_index=[2, 1602], y=[1]),
 Data(x=[90, 266], edge_index=[2, 1603], y=[1]),
 Data(x=[90, 266], edge_index=[2, 1602], y=[1]),
 Data(x=[90, 266], edge_index=[2, 1602], y=[1]),
 Data(x=[90, 266], edge_index=[2, 1602], y=[1]),
 Data(x=[90, 266], edge_index=[2, 1602], y=[1]),
 Data(x=[90, 266], edge_index=[2, 1603], y=[1]),
 Data(x=[90, 266], edge_index=[2, 1602], y=[1]),
 Data(x=[90, 266], edge_index=[2, 1602], y=[1]),
 Data(x=[90, 266], edge_index=[2, 1602], y=[1]),
 Data(x=[90, 266], edge_index=[2, 1602], y=[1]),
 Data(x=[90, 266], edge_index=[2, 1602], y=[1]),
 Data(x=[90, 266], edge_index=[2, 1602], y=[1]),
 Data(x=[90, 266], edge_index=[2, 1602], y=[1]),
 Data(x=[90, 266], edge_index=[2, 1603], y=[1]),
 Data(x=[90, 266], edge_index=[2, 1602], y=[1]),
 Data(x=[90, 266], edge_index=[2, 1602], y=[1]),
 Data(x=[90, 266], edge_index=[2, 1602], y=[1]),
 Data(x=[90, 266], e

In [15]:
train_dataset = data_processed[:500]
test_dataset = data_processed[500:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

Number of training graphs: 500
Number of test graphs: 118


In [16]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 64
DataBatch(x=[5760, 266], edge_index=[2, 102543], y=[64], batch=[5760], ptr=[65])

Step 2:
Number of graphs in the current batch: 64
DataBatch(x=[5760, 266], edge_index=[2, 102543], y=[64], batch=[5760], ptr=[65])

Step 3:
Number of graphs in the current batch: 64
DataBatch(x=[5760, 266], edge_index=[2, 102541], y=[64], batch=[5760], ptr=[65])

Step 4:
Number of graphs in the current batch: 64
DataBatch(x=[5760, 266], edge_index=[2, 102541], y=[64], batch=[5760], ptr=[65])

Step 5:
Number of graphs in the current batch: 64
DataBatch(x=[5760, 266], edge_index=[2, 102545], y=[64], batch=[5760], ptr=[65])

Step 6:
Number of graphs in the current batch: 64
DataBatch(x=[5760, 266], edge_index=[2, 102538], y=[64], batch=[5760], ptr=[65])

Step 7:
Number of graphs in the current batch: 64
DataBatch(x=[5760, 266], edge_index=[2, 102543], y=[64], batch=[5760], ptr=[65])

Step 8:
Number of graphs in the current batch: 52
DataBatch(x=[4680, 266], e

In [17]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool, global_add_pool, global_max_pool
class GAT1(torch.nn.Module):
    def __init__(self):
        super(GAT1, self).__init__()
        self.hid = 16
        self.in_head = 16
        self.out_head = 4
        self.conv1 = GATConv(266, self.hid, heads=self.in_head, dropout=0.5)
        self.conv2 = GATConv(self.hid*self.in_head, 4, concat=False,
                             heads=self.out_head, dropout=0.5)
        self.lin = Linear(4, 2)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        x = self.conv1(x, edge_index)
        x = F.tanh(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)

        # 2. Readout layer
        x = global_max_pool(x, batch)

        # 3. Apply a final classifier
        x = self.lin(x)

        return x


In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = GAT1().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()
def train():
    model.train()
    for data in train_loader:
        data.to(device)
        optimizer.zero_grad()
        out = model(data)
        # print('out: ', out.shape)
        # print('data.y: ', data.y.shape)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()


def test(loader):
    model.eval()

    correct = 0
    for data in loader:
        data.to(device)
        out = model(data)
        pred = out.argmax(dim=1)
        # print('pred: ', pred)
        # print('data.y: ', data.y)
        correct += int((pred == data.y).sum())

    return correct / len(loader.dataset)

for epoch in range(1, 150):
    train()
    # print('train: ')
    train_acc = test(train_loader)
    # print('test: ')
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

cuda
Epoch: 001, Train Acc: 0.5440, Test Acc: 0.4746
Epoch: 002, Train Acc: 0.4600, Test Acc: 0.5169
Epoch: 003, Train Acc: 0.6120, Test Acc: 0.5169
Epoch: 004, Train Acc: 0.5740, Test Acc: 0.5254
Epoch: 005, Train Acc: 0.5980, Test Acc: 0.4831
Epoch: 006, Train Acc: 0.6080, Test Acc: 0.5339
Epoch: 007, Train Acc: 0.5960, Test Acc: 0.5000
Epoch: 008, Train Acc: 0.6200, Test Acc: 0.5169
Epoch: 009, Train Acc: 0.6280, Test Acc: 0.5085
Epoch: 010, Train Acc: 0.6240, Test Acc: 0.5169
Epoch: 011, Train Acc: 0.6200, Test Acc: 0.5000
Epoch: 012, Train Acc: 0.5720, Test Acc: 0.4915
Epoch: 013, Train Acc: 0.6640, Test Acc: 0.5678
Epoch: 014, Train Acc: 0.5540, Test Acc: 0.4746
Epoch: 015, Train Acc: 0.6680, Test Acc: 0.5254
Epoch: 016, Train Acc: 0.5860, Test Acc: 0.4915
Epoch: 017, Train Acc: 0.5900, Test Acc: 0.5085
Epoch: 018, Train Acc: 0.6760, Test Acc: 0.5169
Epoch: 019, Train Acc: 0.6060, Test Acc: 0.5000
Epoch: 020, Train Acc: 0.6180, Test Acc: 0.5000
Epoch: 021, Train Acc: 0.6820, Test